In [1]:
import pandas as pd

In [3]:
coupon = pd.read_excel("http://byungwan.com/class/Coupon_Assignment2.xlsx")
coupon.head()

,CID,Gender,Age,CClass,Discount,Visits,Email,SMS,MStatus,Purchases,Sales
0,1,Male,25,2,Coupon Not Used,1,Opt-in,Opt-in,Married,1,0.5050
1,2,Female,17,2,Coupon Not Used,1,Opt-in,Opt-in,Married,1,0.3300
2,3,Female,38,2,Coupon Not Used,3,Opt-in,Opt-in,Married,1,0.2550
3,4,Female,36,2,Coupon Not Used,6,Opt-in,Opt-in,Single,3,1.8762
4,5,Female,37,2,Coupon Not Used,7,Opt-in,Opt-in,Single,5,6.0940


In [6]:
coupon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CID        1000 non-null   int64  
 1   Gender     1000 non-null   object 
 2   Age        1000 non-null   int64  
 3   CClass     1000 non-null   int64  
 4   Discount   1000 non-null   object 
 5   Visits     1000 non-null   int64  
 6   Email      1000 non-null   object 
 7   SMS        1000 non-null   object 
 8   MStatus    1000 non-null   object 
 9   Purchases  1000 non-null   int64  
 10  Sales      1000 non-null   float64
dtypes: float64(1), int64(5), object(5)
memory usage: 86.1+ KB


In [7]:
coupon2 = pd.get_dummies(coupon, columns=["Gender", "Discount","Email", "SMS","MStatus"], drop_first=True, dtype=int)
coupon2.head()

,CID,Age,CClass,Visits,Purchases,Sales,Gender_Male,Discount_Coupon Used,Email_Opt-out,SMS_Opt-out,MStatus_Single
0,1,25,2,1,1,0.5050,1,0,0,0,0
1,2,17,2,1,1,0.3300,0,0,0,0,0
2,3,38,2,3,1,0.2550,0,0,0,0,0
3,4,36,2,6,3,1.8762,0,0,0,0,1
4,5,37,2,7,5,6.0940,0,0,0,0,1


In [14]:
y = coupon2.iloc[:, 7]

In [15]:
x = coupon2.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10]]

In [16]:
x.head()

,Age,CClass,Visits,Purchases,Sales,Gender_Male,Email_Opt-out,SMS_Opt-out,MStatus_Single
0,25,2,1,1,0.5050,1,0,0,0
1,17,2,1,1,0.3300,0,0,0,0
2,38,2,3,1,0.2550,0,0,0,0
3,36,2,6,3,1.8762,0,0,0,1
4,37,2,7,5,6.0940,0,0,0,1


In [17]:
import statsmodels.api as sm

In [18]:
reg = sm.OLS(y, x).fit()
print(reg.summary())

                                  OLS Regression Results                                 
Dep. Variable:     Discount_Coupon Used   R-squared (uncentered):                   0.133
Model:                              OLS   Adj. R-squared (uncentered):              0.125
Method:                   Least Squares   F-statistic:                              16.89
Date:                  Mon, 13 Oct 2025   Prob (F-statistic):                    4.18e-26
Time:                          23:35:27   Log-Likelihood:                         -38.946
No. Observations:                  1000   AIC:                                      95.89
Df Residuals:                       991   BIC:                                      140.1
Df Model:                             9                                                  
Covariance Type:              nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [21]:
vif_factors = [vif(x.values, i) for i in range(0, x.shape[1])]

In [22]:
x.columns

Index(['Age', 'CClass', 'Visits', 'Purchases', 'Sales', 'Gender_Male',
       'Email_Opt-out', 'SMS_Opt-out', 'MStatus_Single'],
      dtype='object')

In [23]:
pd.DataFrame({"Variable": x.columns, "VIF Factor": vif_factors})

,Variable,VIF Factor
0,Age,8.797168
1,CClass,9.499904
2,Visits,2.497248
3,Purchases,10.060516
4,Sales,7.567112
5,Gender_Male,1.450346
6,Email_Opt-out,1.195162
7,SMS_Opt-out,1.417473
8,MStatus_Single,1.302660


In [24]:
# Most variables had VIF values below 10, but "Purchases" showed a value slightly above 10 
# indicating potential multicollinearity -> make correction

In [26]:
x2 = coupon2.iloc[:, [1, 2, 3, 5, 6, 8, 9, 10]]
x2.head()

,Age,CClass,Visits,Sales,Gender_Male,Email_Opt-out,SMS_Opt-out,MStatus_Single
0,25,2,1,0.5050,1,0,0,0
1,17,2,1,0.3300,0,0,0,0
2,38,2,3,0.2550,0,0,0,0
3,36,2,6,1.8762,0,0,0,1
4,37,2,7,6.0940,0,0,0,1


In [27]:
reg2 = sm.OLS(y, x2).fit()
print(reg2.summary())

                                  OLS Regression Results                                 
Dep. Variable:     Discount_Coupon Used   R-squared (uncentered):                   0.128
Model:                              OLS   Adj. R-squared (uncentered):              0.121
Method:                   Least Squares   F-statistic:                              18.28
Date:                  Mon, 13 Oct 2025   Prob (F-statistic):                    1.09e-25
Time:                          23:41:35   Log-Likelihood:                         -41.534
No. Observations:                  1000   AIC:                                      99.07
Df Residuals:                       992   BIC:                                      138.3
Df Model:                             8                                                  
Covariance Type:              nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------

In [29]:
x2_c = sm.add_constant(x2)

In [30]:
reg3 = sm.OLS(y, x2_c).fit()
print(reg3.summary())

                             OLS Regression Results                             
Dep. Variable:     Discount_Coupon Used   R-squared:                       0.068
Model:                              OLS   Adj. R-squared:                  0.061
Method:                   Least Squares   F-statistic:                     9.091
Date:                  Mon, 13 Oct 2025   Prob (F-statistic):           4.11e-12
Time:                          23:42:26   Log-Likelihood:                -36.980
No. Observations:                  1000   AIC:                             91.96
Df Residuals:                       991   BIC:                             136.1
Df Model:                             8                                         
Covariance Type:              nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.119

In [31]:
residual = reg3.resid

In [32]:
exog = reg3.model.exog

In [33]:
bp_test = sm.stats.het_breuschpagan(residual, exog)
labels = ["Test Statistics","p-value", "f statistics", "f p-value"]

In [34]:
list(zip(labels, bp_test))

[('Test Statistics', np.float64(70.334408105023)),
 ('p-value', np.float64(4.215839896308745e-12)),
 ('f statistics', np.float64(9.37183744345137)),
 ('f p-value', np.float64(1.5658723804270076e-12))]

In [35]:
# The test shows the presence of heteroskedasticity -> needs correction

In [36]:
reg4 = sm.OLS(y, x2_c).fit(cov_types="HC3")
print(reg4.summary())

                             OLS Regression Results                             
Dep. Variable:     Discount_Coupon Used   R-squared:                       0.068
Model:                              OLS   Adj. R-squared:                  0.061
Method:                   Least Squares   F-statistic:                     9.091
Date:                  Mon, 13 Oct 2025   Prob (F-statistic):           4.11e-12
Time:                          23:45:24   Log-Likelihood:                -36.980
No. Observations:                  1000   AIC:                             91.96
Df Residuals:                       991   BIC:                             136.1
Df Model:                             8                                         
Covariance Type:              nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.119

[Interpretation]

The dependent variable is Discount_Coupon Used indicating whether a customer used the coupon or not.
After removing the multicollinear variable "Purchases" and applying the HC3 error correction to address heteroskedasticity the final regression results are as follows :

R-squared = 0.068 -> The model explains about 6.8% of the variation in coupon usage which is relatively low.

According to the p-value, Age, Visits, Email, SMS, MStatus are not statistically significant. (p-value>0.05)
This implies that coupon usage behavior is not strongly related to age, visit frequency, or marital status once other variables are controlled.

The regression results show that constant, customer class (CClass), Sales, and Gender significantly influence coupon usage.

To be specific, when a customer's class level (CClass) increases by one unit, the likelihood of using a coupon increases by 0.124 units, holding other variables constant. In contrast, for every one-unit increase in Sales, coupon usage decreases by 0.025 units suggesting that customers who spend more are less motivated by discounts.
Additionally, being male increases coupon usage by approximately 0.037 units compared to females indicating a modest but significant gender difference in discount behavior.

Overall, the model suggests that coupon campaigns may be most effective when targeted toward loyal but price-sensitive customers rather than high-spending ones.